In [54]:
import osmnx as ox
import geopy
from geopy.geocoders import Nominatim
from geopy.distance import geodesic as geopy_distance
from geopy.extra.rate_limiter import RateLimiter
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import networkx as nx
import matplotlib.pyplot as plt

In [55]:
import random
# Define a function to generate random coordinates within a given radius of a central point
def generate_random_coordinates(center_lat, center_lon, radius):
    # Generate random distance and angle
    random_distance = random.random() * radius
    random_angle = random.random() * 360
    # Calculate random point coordinates using geopy.distance
    return geopy_distance(kilometers=random_distance).destination((center_lat, center_lon), bearing=random_angle)


# Initialize geolocator with a user-agent
geolocator = Nominatim(user_agent="routing-system-big-data")

# List of areas to geocode
areas = ["Drumul Taberei, Bucharest", "Berceni, Bucharest", "Colentina, Bucharest", "Militari, Bucharest"]
radius = 2  # Radius in kilometers within which to generate house locations
num_locations_per_area = 1  # Number of locations to generate per area

# Get central coordinates of the areas
central_coordinates = {area: geolocator.geocode(area) for area in areas}
# Generate house locations
house_locations = []
for area, location in central_coordinates.items():
    if location:
        for _ in range(num_locations_per_area):
            random_coords = generate_random_coordinates(location.latitude, location.longitude, radius)
            house_locations.append((random_coords[0], random_coords[1]))

# house_locations now contains the generated coordinates for the house locations
house_locations

[(44.428667183390466, 26.02398683113146),
 (44.388716342433526, 26.11845793684689),
 (44.45780567347374, 26.143310728777045),
 (44.43128209932741, 26.011884727901315)]

In [56]:
# Constants for the routing problem
CENTER_LOCATION = (44.413162, 26.163739)  # Replace with actual depot coordinates
DIST = 3000  # Adjust as needed
NUM_VEHICLES = 2  # Adjust as needed

# Get the road network graph
G = ox.graph_from_point(CENTER_LOCATION, dist=DIST, network_type='drive')
G = ox.utils_graph.get_largest_component(G, strongly=True)

# Function to calculate distance matrix using networkx
def calculate_distance_matrix(locations, G):
    distance_matrix = []
    for loc1 in locations:
        row = []
        for loc2 in locations:
            node1 = ox.nearest_nodes(G, loc1[1], loc1[0])
            node2 = ox.nearest_nodes(G, loc2[1], loc2[0])
            length = round(nx.shortest_path_length(G, node1, node2, weight='length'))
            row.append(length)
        distance_matrix.append(row)
    return distance_matrix


# Add depot to the list of locations and calculate the distance matrix
all_locations = [CENTER_LOCATION] + house_locations
distance_matrix = calculate_distance_matrix(all_locations, G)


In [57]:
distance_matrix

[[0, 4319, 3886, 4843, 3974],
 [4144, 0, 5768, 2771, 885],
 [3906, 6044, 0, 6568, 5700],
 [4992, 2811, 6617, 0, 2226],
 [4059, 755, 5684, 2261, 0]]

In [59]:
# Create and configure the routing model
def create_model(distance_matrix, num_vehicles):

    manager = pywrapcp.RoutingIndexManager(len(distance_matrix), num_vehicles, 0)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):

        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]


    transit_callback_index = routing.RegisterTransitCallback(distance_callback) 

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(transit_callback_index,
                          0,
                          30000, # max distance per vehicle
                          True,
                          dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    return routing, manager

# Solve the routing problem
def solve_routing_problem(routing, manager):
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        print_solution(solution, routing, manager)
    else:
        print("No solution found!")

# Print the solution
def print_solution(solution, routing, manager):
    print("Routes:")
    total_distance = 0
    for vehicle_id in range(manager.GetNumberOfVehicles()):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id+1)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} ->'.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        plan_output += ' {}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        total_distance += route_distance
    print('Total Distance of all routes: {}m'.format(total_distance))

# Now use these functions with your data

routing, manager = create_model(distance_matrix, NUM_VEHICLES)
solve_routing_problem(routing, manager)

Routes:
Route for vehicle 1:
 0 -> 2 -> 0
Distance of the route: 7792m

Route for vehicle 2:
 0 -> 3 -> 4 -> 1 -> 0
Distance of the route: 11968m

Total Distance of all routes: 19760m
